In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

db = chromadb.PersistentClient(path="../Storing/chroma")

# collection_name = "E9_VISA_GUIDE_AND_FAQ"
collection_name = "E9_VISA_GUIDE"

chroma_collection = db.get_or_create_collection(collection_name)
chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store)

In [5]:
merge_retriever = index.as_retriever(similarity_top_k=5)

In [22]:
def pretty(retrievals):
    for r in retrievals:
        print("-----------------------------------------------------------------------------------------\n")
        print(f"Page: {r.metadata['page_label']}")
        print(f"Score: {r.score:.3f}")
        print(f"{r.get_content()}")
        print("-----------------------------------------------------------------------------------------\n\n")

In [20]:
retrievals= merge_retriever.retrieve("고용허가서 신청 절차는 어떻게 되나요?")
pretty(retrievals)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
-----------------------------------------------------------------------------------------

Page: 15
Score: 0.737
고용허가제 해당자가 근무처(직장)를 변경하는 절차 및 제출 서류
        ● 근무처(직장) 변경 절차
            ○ 사용자와 근로계약 종료 후 1월 이내에 사업장 변경을 신청(고용노동부 고용센터)하여야 하며, 사업장 변경을 신청한 날로부터 3개월 이내에 근무처 변경허가(관할 출입국·외국인청(사무소ᆞ출장소))를 받아야 함
            ○ 다만, 업무상 재해, 질병, 임신, 출산 등의 사유로 근무처변경 허가를 받을 수 없거나 신청할 수 없는 경우에는 사유해소일로부터 기간 계산
            ○ 사유 해당자는 체류기간 만료 전에 직업안정기관의 장이 발급한 사업장변경신청기간 연장접수확인서에 산재입증서류, 진단서 등을 첨부하여 관할 출입국ᆞ외국인청(사무소ᆞ출장소)에 체류기간연장을 신청
            ○ 체류기간연장허가 수수료 면제
            ○ 체류기간연장허가를 받은 기간 중에라도 사유가 해소되어 직업안정기관으로부터 사업장변경허가를 받은 경우에는 관할 출입국ᆞ외국인청(사무소ᆞ출장소)장의 근무처변경허가를 받은 후에 취업을
            개시하여야 함
            ○ 체류기간 연장허가 상한은 최초 입국일로부터 3년, 재고용에 의한 취업기간 연장 허가기간 중에는 최초 입국일로부터 4년 10개월을 초과할 수 없음
        ● 근무처(직장) 변경 시 제출 서류
            ○ 신청서(별지 34호서식), 여권 및 외국인등록

## Configure chat engine

Since the context retrieved can take up a large amount of the available LLM context, let's ensure we configure a smaller limit to the chat history!

In [23]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

reranker = CohereRerank(
    top_n=2, model="rerank-multilingual-v2.0"
)

In [35]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.llms.openai import OpenAI

memory = ChatMemoryBuffer.from_defaults(token_limit=7000)

prompt = (
    "You are a chatbot, able to have normal interactions, as well as talk"
    " about E-9 Visa Guide"
    "Here are the relevant documents for the context:\n"
    "{context_str}"
    "\nInstruction: Use the previous chat history, or the context above, to interact and help the user about Korea Visa."
    "\nYou MUST NOT use prior knowledge for answer the question."
)

# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4-0613")

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=prompt,
    similarity_top_k=2,
    # node_postprocessors=[reranker],
    verbose=False,
)

# chat_engine = index.as_chat_engine(
#     chat_mode="condense_question", 
#     llm=llm,
#     context_prompt=prompt,
#     similarity_top_k=3,
#     # node_postprocessors=[reranker],
#     verbose=True
# )

In [46]:
response = chat_engine.stream_chat("서비스업에 재료수집업이 있다고 했는데 재료수집업이라는게 쉽게 설명해서 어떤 일을 하는건가요?")

response.print_response_stream()
# print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: 재료수집업이란 어떤 일을 하는 것인가요?
Condensed question: 재료수집업이란 어떤 일을 하는 것인가요?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


ValueError: Initial token count exceeds token limit

In [47]:
for node in response.source_nodes:
    print(f"metadata: {node.metadata['page_label']}")
    print(node.get_content())

metadata: 5
사증발급 허용업종 및 체류자격 약호(기호): E-9 근로자를 채용할 수 있는 업종과 범위
        ● 제조업: 제조업, 뿌리산업
            ○ 체류자격: E-9-1
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 상시근로자 300인 미만 또는 자본금 80억원 이하 제조업
                ■ 단, 상기 기준에 충족하지 않더라도 아래 증빙서류 제출 시 인정 : 중소기업(지방중소기업청에서 발급한 ‘중소기업확인서’) / 비수도권 소재 
                뿌리산업 중견기업 (한국생산기술연구원에서 발급한 ‘뿌리기업 확인서’ 및 한국중견기업연합회에서 발급한 ‘중견기업 확인서’)
        ● 건설업
            ○ 체류자격: E-9-2
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 모든 건설공사
                ■ ※ 발전소·제철소·석유화학 건설현장의 건설업체 중 건설면허가 산업환경 설비인 경우에는 적용 제외
        ● 농축산업
            ○ 체류자격: E-9-3
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 농업: 작물재배업
                ■ 축산업
                ■ 농축산서비스업: 작물재배 및 축산관련 서비스업
        ● 어업
            ○ 체류자격: E-9-4
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 연근해어업: 연안어업ᆞ근해어업
                    ● 선원법 적용을 받지 않는 20톤 미만 어선 종사자에 한함
                ■ 양식어업
                ■ 소금채취업
        ● 임업
            ○ 체류자격:
          